<a href="https://colab.research.google.com/github/WK-Chen/MAIR_23/blob/main/classifier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    print ('saved file', name)

Saving dialog_acts.dat to dialog_acts.dat
saved file dialog_acts.dat


In [ ]:
import csv

# Specify input and output file names
input_file = 'dialog_acts.dat'
output_file = 'dialog_acts.csv'

# Initialize a list to store the data
data = []

# Read the input file and split lines into tag and sentence
with open(input_file, 'r') as file:
    for line in file:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            tag, sentence = parts
            data.append([tag, sentence])

# Write the data to a CSV file
with open(output_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write a header row if needed
    # csvwriter.writerow(['Tag', 'Sentence'])  # Uncomment this line if you want a header row

    # Write the data rows
    csvwriter.writerows(data)

print(f"Conversion complete. Data saved in '{output_file}'.")


In [7]:
label_dic = {
"ack": 0,
"affirm": 1,
"bye": 2,
"confirm": 3,
"deny": 4,
"hello": 5,
"inform": 6,
"negate": 7,
"null": 8,
"repeat": 9,
"reqalts": 10,
"reqmore": 11,
"request": 12,
"restart": 13,
"thankyou": 14,
}

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Define your dataset class
class DSTCDataset(Dataset):
    def __init__(self, path, tokenizer, dataset='trn'):
      self.data = self.process(path, dataset)
      self.tokenizer = tokenizer

    def process(self, path, dataset):
      # Copy data from Raluca, this part should be a independent function
      with open('dialog_acts.csv', 'r') as file:
        lines = file.readlines()
      data = []
      for line in lines:
        label, sentence = line.split(",")
        data.append({"text":sentence, "label":label})
      if dataset == 'trn':
        data = data[:int(len(data)*0.85)]
      elif dataset == 'test':
        data = data[int(len(data)*0.85):]
      else:
        assert False
      return data

    def __len__(self):
      return len(self.data)

    def __getitem__(self, idx):
      item = self.data[idx]
      text = item['text']
      labels = label_dic[item['label']]
      encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=64, return_tensors='pt')

      inputs = {
          'input_ids': encoding['input_ids'].squeeze(),
          'attention_mask': encoding['attention_mask'].squeeze(),
          'labels': labels
      }

      return inputs


# Load your pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=15)

# Define hyperparameters
batch_size = 16
learning_rate = 2e-5
epochs = 5

# Create data loaders for training and validation
train_dataset = DSTCDataset("dialog_acts.csv", tokenizer)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = DSTCDataset("dialog_acts.csv", tokenizer, 'test')
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {average_loss:.4f}")

# Validation loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch in tqdm(validation_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predicted = torch.argmax(outputs.logits, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy:.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 48/48 [05:37<00:00,  7.04s/it]


Epoch 1/1, Loss: 1.8690


100%|██████████| 32/32 [01:09<00:00,  2.17s/it]

Validation Accuracy: 0.8611
